In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# before this step, connect with google drive
%cd /content/drive/MyDrive/cs679_project_compare_40/code/model_speechbrain

/content/drive/MyDrive/cs679_project_compare_40/code/model_speechbrain


Metadata: [filename, speaker embedding, spectrogram]

Modify Speaker Embedding

In [3]:
import glob
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
import librosa
import torch.nn as nn
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

In [6]:
def extract_embedding(audio_path, hidden_size=192, output_embedding_size=256):
    signal, fs =torchaudio.load(audio_path)
    signal=signal.to(device)
    assert fs == 16000, "The sample rate of the audio must be 16000."
    embeddings = classifier.encode_batch(signal).to(device)
    original_embedding_size = hidden_size
    desired_embedding_size = output_embedding_size
    linear_transform = nn.Linear(original_embedding_size, desired_embedding_size).to(device)
    embeddings = linear_transform(embeddings.squeeze())
    return embeddings.cpu()

In [9]:
from tqdm import tqdm
def embedding_each_wav(folder_path):
    all_embeddings = []
    for audio_path in tqdm(glob.glob(folder_path + '*.wav')):
        embedding = extract_embedding(audio_path)
        all_embeddings.append(embedding)
    all_embeddings_tensor = torch.stack(all_embeddings)
    average_embedding = all_embeddings_tensor.mean(dim=0)
    return average_embedding

Modify Spectrogram

In [12]:
import pandas as pd
import numpy as np
import pickle

In [15]:
df = pd.read_pickle('original/metadata.pkl')
metadata_0 = [df[i][0] for i in range(len(df))]
#wav_names = [name+'x'+name+'.wav' for name in metadata_0]
folder_names = ['wavs/'+name+'/' for name in metadata_0]
folder_names

['wavs/p225/',
 'wavs/p226/',
 'wavs/p227/',
 'wavs/p228/',
 'wavs/p229/',
 'wavs/p230/',
 'wavs/p231/',
 'wavs/p232/',
 'wavs/p233/',
 'wavs/p234/',
 'wavs/p236/',
 'wavs/p237/',
 'wavs/p238/',
 'wavs/p239/',
 'wavs/p240/',
 'wavs/p241/',
 'wavs/p243/',
 'wavs/p244/',
 'wavs/p245/',
 'wavs/p246/',
 'wavs/p247/',
 'wavs/p248/',
 'wavs/p249/',
 'wavs/p250/',
 'wavs/p251/',
 'wavs/p252/',
 'wavs/p253/',
 'wavs/p254/',
 'wavs/p255/',
 'wavs/p256/',
 'wavs/p257/',
 'wavs/p258/',
 'wavs/p259/',
 'wavs/p260/',
 'wavs/p261/',
 'wavs/p262/',
 'wavs/p263/',
 'wavs/p264/',
 'wavs/p265/',
 'wavs/p266/']

In [ ]:
for i in range(len(df)):
    df[i][1] = embedding_each_wav(folder_names[i]).detach().numpy().reshape(-1)
    with open('temp.pkl', 'wb') as file:
        pickle.dump(df, file)

In [ ]:
# with open('metadata_lstm.pkl', 'wb') as file:
#     pickle.dump(df, file)

In [ ]:
with open('metadata_speechbrain.pkl', 'wb') as file:
    pickle.dump(df, file)

In [ ]:
df = pd.read_pickle('metadata_speechbrain.pkl')
df[0][1].shape

In [ ]:
df[0][1]